In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150
from scipy.io import savemat
from tenpy.tools.params import get_parameter
from matplotlib import colors
import matplotlib.pyplot as plt
import sys
from tenpy.models.mixed_xk import MixedXKLattice

/Users/gleb/TeNPy/tenpy/tools/optimization.py:307: UserWarning: Couldn't load compiled cython code. Code will run a bit slower.
  warnings.warn("Couldn't load compiled cython code. Code will run a bit slower.")


In [2]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS
from tenpy.models.tf_ising import TFIChain
tenpy.tools.misc.setup_logging(to_stdout="INFO")
import pickle
from tenpy.tools.misc import to_array, inverse_permutation, to_iterable

In [3]:
from tenpy.networks.mps import MPS
from tenpy.models.tf_ising import TFIChain
from tenpy.algorithms import dmrg
from tenpy.networks.site import SpinSite, SpinHalfSite, SpinHalfFermionSite, FermionSite
from tenpy.models.lattice import Triangular, Square
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel

In [4]:
''' Fermi-Hubbard model '''

class Hubbard_mixed(CouplingModel, MPOModel):
    def __init__(self, model_param):
        
        ''' system size '''
        Lx = model_param["Lx"]
        Ly = model_param["Ly"]
#         Lx = q 
        
        ''' coupling constants'''
        t = model_param["t"]
        V = model_param["V"]
#         phi = model_param["phase"]
        
        
        ''' boundary conditions'''
        bc_MPS = model_param["bc_MPS"]
        bc_y = model_param["bc_y"]
        bc_x = model_param["bc_x"]
        
        
        ''' site with particle + U(1) symmetry conservation'''
        site = FermionSite(conserve='N', filling = 0.5)
        
        ''' define square lattice'''
        lat = Square(Lx, Ly, site, bc=[bc_x, bc_y], bc_MPS=bc_MPS)   
        

        CouplingModel.__init__(self, lat)
        
        dR1 = [1,0]
        
        if model_param["flux"]:
            
            '''hopping terms'''
            hops_along_x = (-t) * np.array([-1j + (-1) * np.exp(-1j * 2 * np.pi/Ly * np.arange(Ly)), -1j + np.exp(-1j * 2 * np.pi/Ly * np.arange(Ly))])
            hops_y = (-t) * np.array([2 * np.cos(2 * np.pi/Ly * np.arange(Ly)), - 2 * np.cos(2 * np.pi/Ly * np.arange(Ly))])
            
            self.add_coupling(hops_along_x, 0, 'Cd', 0,'C', dR1, plus_hc=True)
            self.add_onsite(hops_y, 0, 'N')
         
        else:
            
            '''hopping terms'''
            hops_along_x = (-t) * np.array([1 + (+1) * np.exp(-1j * 2 * np.pi/Ly * np.arange(Ly))])
            hops_y = (-t) * np.array([2 * np.cos(2 * np.pi/Ly * np.arange(Ly))])
            self.add_coupling(hops_along_x, 0, 'Cd', 0,'C', dR1, plus_hc=True)
            self.add_onsite(hops_y, 0, 'N')
        
        ''' density-density interaction'''
        for q1 in range(Ly):
            for q2 in range(Ly):
                dx1 = [0, q1]
                dx2 = [0, 0]
                dx4 = [0, q2]
                dx3 = [0, q2 - q1]

                self.add_multi_coupling(V/Ly, [('Cd', [0,q1], 0), ('C', [0,0], 0), ('Cd', [1,q2-q1], 0), ('C', [1,q2], 0)])
                self.add_multi_coupling(np.exp(-1j * 2 * np.pi/Ly * q1) * V/Ly, [('Cd', [0,q1], 0), ('C', [0,0], 0), ('Cd', [1,q2-q1], 0), ('C', [1,q2], 0)])

                if q1 == 0 and q2 == 0:
                    dd_interaction = [[V/Ly] * Ly, [V/Ly] * Ly]
                    self.add_onsite(dd_interaction, 0, 'N')

                else:
                    self.add_multi_coupling(np.cos(2 * np.pi/Ly * q1) * V/Ly, [('Cd', dx1, 0), ('C', dx2, 0), ('Cd', dx3, 0), ('C', dx4, 0)])                    
        
        MPOModel.__init__(self, lat, self.calc_H_MPO())
#         ax = plt.gca()
#         lat.plot_basis(ax)

In [5]:
# if flux = True, Lx = 2
# if flux = False, Lx = 1

model_param = {"Lattice" : Square,
              "Ly" : 3,
              "Lx" : 2,
              "t" : 1.0,
              "V" : 4.0,
              "fraction" : 2,
              "bc_MPS" : "infinite",
              "bc_y" : 'periodic',
              "bc_x" : 'periodic',
              "flux" : True}

# chi_list = tenpy.algorithms.dmrg.chi_list(500, dchi=200, nsweeps=20)
chi_list = {0:200, 20:500, 40:500}
dmrg_params = {"trunc_params": {"chi_max": 500, "svd_min": 1.e-10}, "mixer": True, "chi_list" : chi_list,"max_sweeps": 100}

In [6]:
M = Hubbard_mixed(model_param)
sites = M.lat.mps_sites()
psi = MPS.from_product_state(sites,["full","full","full","empty","empty","empty"],"infinite")

In [7]:
info = dmrg.run(psi, M, dmrg_params)
energy = info['E']
delta_energy = abs(info["sweep_statistics"]['E'][-1] - info["sweep_statistics"]['E'][-2])

INFO    : DMRG: subconfig 'trunc_params'=Config(<2 options>, 'trunc_params')
INFO    : DMRG: reading 'chi_list'={0: 200, 20: 500, 40: 500}
INFO    : start environment_sweep
INFO    : trunc_params: reading 'chi_max'=500
INFO    : trunc_params: reading 'svd_min'=1e-10
INFO    : DMRG: reading 'max_sweeps'=100
INFO    : DMRG: reading 'mixer'=True
INFO    : activate DensityMatrixMixer with initial amplitude 1.0e-05
INFO    : Running sweep with optimization
INFO    : activate DensityMatrixMixer with initial amplitude 1.0e-05
INFO    : Setting chi_max=200
INFO    : start environment_sweep
INFO    : checkpoint after sweep 10
energy=1.1988363260860335, max S=1.3969898841408552, age=198, norm_err=2.7e-09
Current memory usage 218836.0MB, wall time: 289.4s
Delta E = nan, Delta S = 9.8435e-02 (per sweep)
max trunc_err = 2.2904e-09, max E_trunc = 2.7593e-08
chi: [200, 200, 200, 200, 200, 200]
INFO    : Running sweep with optimization
INFO    : disable mixer after 15 sweeps, final amplitude 3.05e-10


In [8]:
# Ly =6, Lx = 2, t = 1, U = 3, chi_max = 500, E = -0.1962525089224651 Delta E = 8.5273e-08

In [9]:
# SPINLESS MIXED
# t = 1.57, U =0, ["full","full","empty","empty","empty","empty"]: 
# E = -1.370900, Delta_E = -1.6202e-03